In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# PDF Load

In [ ]:
# PDF Loading
from langchain.document_loaders import PyPDFLoader
pdf_files = ["27_Meditation_Techniques.pdf", "how-to-meditate.pdf"]
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)
documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs.")

Loaded 39 documents from 2 PDFs.


In [ ]:
len(documents)

39

# Text Split

In [ ]:
# Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 89 chunks.


# Embedding

In [ ]:
# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Vector Store

In [ ]:
# Vectorstore (ChromaDB)
from langchain_chroma import Chroma as ch
persist_directory = "chroma_db"
db = ch.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

# Retriever

In [ ]:
# Retriever with similarity score threshold
retriever = db.as_retriever(
    search_type="similarity", #change
    search_kwargs={"k": 4}
)

In [ ]:
# RAG Prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG LCEL chain
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

/tmp/ipython-input-9-3785023118.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
rag_chain.invoke({"question": "What is meditation?"})

'Meditation is commonly described as a training of mental attention that awakens us beyond the conditioned mind and habitual thinking, and reveals the nature of reality. It is understood as Natural Presence, which is a mindful, clear recognition of what is happening—here, now—and the open, allowing space that includes all experience.'

In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, so that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you breathed in. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in Chakras**: Breathe in and hold the breath in the third eye for a couple of seconds, then in the heart center the next time you breathe in.\n\n3. **Exhaling Undivine Thoughts**: As you breathe out, visualize exhaling all negative thoughts and impurities.\n\n4. **One-Four-Two Breathing**: Inhale while repeating a name or mantra once, hold your breath while repeating it four times, and exhale while repeating it twice.\n\n5. **Gradual Counts**: You can gradually increase your counts, such as breathing in for four counts, holding for sixteen, and breathing out for eight.\n\n6. **Alternate Nostril 

# RAG Agent 2

In [ ]:
from langchain.tools import tool
from typing import List
import requests
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chat_models import ChatOpenAI

# RAG Prompt Template
custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG Chain
rag_chain = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Define tool
@tool
def rag_tool(question: str) -> str:
    """
    Answers a question using a retrieval-augmented generation (RAG) approach.
    Retrieves relevant documents from a vector store and uses an LLM to generate the final answer.
    """
    return rag_chain.invoke({"question": question})


In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, so that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in Chakras**: Breathe in and hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation.\n\n3. **Exhaling Undivine Thoughts**: As you breathe out, visualize exhaling all negative thoughts and impurities from your system.\n\n4. **One-Four-Two Breathing**: Inhale while repeating a name or mantra once, hold your breath while repeating it four times, and exhale while repeating it twice.\n\n5. **Gradual Counts**: You can practice breathing in for four counts, holding for sixteen, and breathing out for eight, but do this gradually.\n\n6. **Alternate No

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

key = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")
meditation_assistant = create_react_agent(
    model=model,
    tools=[rag_tool],
    prompt="""
You are a helpful Meditation assistant. Your job is to use the RAG tool to answer questions using the provided PDF documents.

Always return your final answer in **valid JSON** format structured like this:
{
  "answer": "Your main answer to the user's query.",
  "source_documents": [
    {
      "title": "Document Title or File Name",
      "page": "Page number or location",
      "snippet": "Short snippet from the document that supports the answer"
    },
    ...
  ]
}

Ensure the JSON is syntactically valid and can be parsed by a machine.
""",
    name="meditation_assistant"
)


In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to do breathing exercises?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to do breathing exercises?', additional_kwargs={}, response_metadata={}, id='76c560c9-6996-4a06-bba2-297e05d58be2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lq2QEk0kjVtnjmBEfSViffzR', 'function': {'arguments': '{"question":"How to do breathing exercises?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 199, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkoKAm9taL8IGgk8OrbGO3MzZD3FR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--49f70df5-5791-49bb-a0d1-b1cf1bf7c458-0', tool_calls=[{'name': 'rag_tool'

In [ ]:
response["messages"][-1].content

'{\n  "answer": "To do breathing exercises, you can follow these steps: 1. **Proper Breathing**: Inhale slowly and quietly, ensuring not to disturb a thread placed in front of your nose. Exhale even more slowly, and if comfortable, hold your breath for a few seconds. 2. **Life-Energy in Chakras**: Hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation. 3. **Exhaling Undivine Thoughts**: Visualize exhaling all negative thoughts as you breathe out. 4. **One-Four-Two Breathing**: Inhale for one count while repeating a mantra, hold for four counts, and exhale for two counts. 5. **Gradual Increase**: Increase your counts gradually over time. 6. **Alternate Nostril Breathing**: Press one nostril, inhale through the other, hold, and then switch nostrils. These exercises are easy and effective, ensuring lung safety.",\n  "source_documents": [\n    {\n      "title": "Breathing Techniques Document",\n      "page": "1",\n      "snippet": "

In [ ]:
import json
response_content = response["messages"][-1].content
# Parse content
parsed = json.loads(response_content)

# Access the answer
answer = parsed["answer"]
print("Answer:", answer)

Answer: To do breathing exercises, you can follow these steps: 1. **Proper Breathing**: Inhale slowly and quietly, ensuring not to disturb a thread placed in front of your nose. Exhale even more slowly, and if comfortable, hold your breath for a few seconds. 2. **Life-Energy in Chakras**: Hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation. 3. **Exhaling Undivine Thoughts**: Visualize exhaling all negative thoughts as you breathe out. 4. **One-Four-Two Breathing**: Inhale for one count while repeating a mantra, hold for four counts, and exhale for two counts. 5. **Gradual Increase**: Increase your counts gradually over time. 6. **Alternate Nostril Breathing**: Press one nostril, inhale through the other, hold, and then switch nostrils. These exercises are easy and effective, ensuring lung safety.


In [ ]:
print(answer)

To do breathing exercises, you can follow these steps: 1. **Proper Breathing**: Inhale slowly and quietly, ensuring not to disturb a thread placed in front of your nose. Exhale even more slowly, and if comfortable, hold your breath for a few seconds. 2. **Life-Energy in Chakras**: Hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation. 3. **Exhaling Undivine Thoughts**: Visualize exhaling all negative thoughts as you breathe out. 4. **One-Four-Two Breathing**: Inhale for one count while repeating a mantra, hold for four counts, and exhale for two counts. 5. **Gradual Increase**: Increase your counts gradually over time. 6. **Alternate Nostril Breathing**: Press one nostril, inhale through the other, hold, and then switch nostrils. These exercises are easy and effective, ensuring lung safety.


# Eleven Labs

In [ ]:
!pip install elevenlabs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.1/740.1 kB 7.0 MB/s eta 0:00:00


In [ ]:
from elevenlabs.client import ElevenLabs

client = ElevenLabs(
  api_key='key here',
)


In [ ]:
client.text_to_speech.convert(
    voice_id="bIQlQ61Q7WgbyZAL7IWj",
    output_format="mp3_44100_128",
    text=answer,
    model_id="eleven_multilingual_v2",
)


<generator object TextToSpeechClient.convert at 0x7b1dc82c8610>

# Audio 1

In [ ]:

# Get generator from TTS
audio_stream = client.text_to_speech.convert(
    voice_id="bIQlQ61Q7WgbyZAL7IWj",
    output_format="mp3_44100_128",
    text=answer,
    model_id="eleven_multilingual_v2",
)

# Save the stream to a file
output_path = "meditation_speech.mp3"
with open(output_path, "wb") as f:
    for chunk in audio_stream:
        f.write(chunk)

print(f"Audio saved to: {output_path}")

Audio saved to: meditation_speech.mp3


# Audio 2

In [ ]:
text="""Begin by sitting comfortably. Gently close your eyes and take a deep breath in... and slowly exhale.

Let your hands rest softly in your lap or by your sides.
Allow your body to settle and your shoulders to relax.

Take another slow breath in through your nose…
and gently breathe out through your mouth.

Now, bring your attention to your body.
Notice the sensations in your feet, your legs, your hands, your chest.
Don’t try to change anything — just observe.

If your mind starts to wander, that’s okay.
Just gently return your attention to your breath —
Inhale…
Exhale…

Feel the air entering and leaving your body.
Notice the rise and fall of your chest.
Let your breath anchor you to this moment.

Now silently repeat to yourself:
“I am here. I am calm. I am safe.”

Feel that calm spread through your body —
like a gentle wave washing over you, easing any tension.

Take one more deep breath…
and slowly exhale.

When you're ready, gently wiggle your fingers and toes.
Bring your awareness back to the room.
And when it feels right, softly open your eyes.

You’ve just given yourself 5 minutes of peace."""

In [ ]:
# Get generator from TTS with custom voice settings
audio_stream = client.text_to_speech.convert(
    voice_id="bIQlQ61Q7WgbyZAL7IWj",
    output_format="mp3_44100_128",
    text=text,
    model_id="eleven_multilingual_v2",
    voice_settings={
        "speed": 0.8
    }
)

# Save the stream to a file
output_path = "meditation_speech_1.mp3"
with open(output_path, "wb") as f:
    for chunk in audio_stream:
        f.write(chunk)

print(f"Audio saved to: {output_path}")


Audio saved to: meditation_speech_1.mp3
